In [1]:
import yfinance as yf
import pandas as pd
import datetime
import pandas_datareader as web
import matplotlib.pyplot as plt
import requests
import time
import numpy as np
import os
import parquet
import pyarrow
import requests

In [2]:
api_key = "TULH49F5XQMU8MQ5"
base_url = "https://www.alphavantage.co/query"

# first we must create a function that retrieves the important data that
# we need. get_fin_stmt will be the basis of our process, which uses the 
# keys in the API to retrieve json data back that has been arranged by analysts.
# the data has been accurate when checked against 10k data for several stocks,
# but the values may be off. This is something to keep in mind and check 
# occasionally.

def get_fin_stmt(symbol, function):
    url = base_url + "?function=" + function + "&symbol=" + symbol + "&apikey=" + api_key
    response = requests.get(url)
    time.sleep(16)
    if response.status_code != 200:
        print("Error, cannot retrieve" + function + "for " + symbol)
        return None
    data = response.json()
    if "annualReports" not in data:
        print("Error: No " + function + " data found for " + symbol)
        return None
#     print(data)
    statement_data = data["annualReports"]
    return_dct = {}
    
    return statement_data

In [3]:
url = "https://www.ishares.com/us/products/239710/ishares-russell-2000-etf/1467271812596.ajax?fileType=csv&fileName=IWM_holdings&dataType=fund"

df = pd.read_csv(url, skiprows=9, usecols=[0])

# Extract the tickers from the dataframe and return them as a list
ticker_list = df.iloc[:, 0].str.upper().tolist()

ticker_list

['SWAV',
 'APLS',
 'INSP',
 'IRDM',
 'EME',
 'SAIA',
 'TXRH',
 'KNSL',
 'CROX',
 'KRTX',
 'RXDX',
 'LNTH',
 'STAG',
 'RBC',
 'SMCI',
 'SIGI',
 'WING',
 'AQUA',
 'CELH',
 'CHRD',
 'ADC',
 'ITCI',
 'MUSA',
 'LNW',
 'NOVT',
 'SPSC',
 'CHX',
 'RLI',
 'SSD',
 'GTLS',
 'FIX',
 'MEDP',
 'CMC',
 'RHP',
 'ISEE',
 'MUR',
 'RMBS',
 'ATKR',
 'ALKS',
 'CIVI',
 'TRNO',
 'MTDR',
 'UFPI',
 'ENSG',
 'AIT',
 'EXLS',
 'TRTN',
 'NJR',
 'BRBR',
 'MMS',
 'SSB',
 'MMSI',
 'ATI',
 'HQY',
 'WTS',
 'ASO',
 'MTH',
 'CWST',
 'ESNT',
 'OPCH',
 'APG',
 'TMHC',
 'POR',
 'DEN',
 'EXPO',
 'ABG',
 'KRG',
 'ORA',
 'POWI',
 'AJRD',
 'HLI',
 'QLYS',
 'BIPC',
 'PCVX',
 'OGS',
 'TGTX',
 'SLAB',
 'FELE',
 'BKH',
 'WFRD',
 'ELF',
 'AAON',
 'WK',
 'ONTO',
 'LANC',
 'MLI',
 'ARWR',
 'PNM',
 'NSP',
 'HAE',
 'HALO',
 'TENB',
 'BMI',
 'FOXF',
 'AMN',
 'PBF',
 'BCPC',
 'GATX',
 'PTCT',
 'LTHM',
 'SHLS',
 'ACLS',
 'FCFS',
 'NSIT',
 'RDN',
 'SFM',
 'HOMB',
 'FLR',
 'MDGL',
 'IRT',
 'BOX',
 'CBT',
 'SKY',
 'UBSI',
 'PCH',
 'IRTC',
 'S

In [4]:
# limit number used in this project to 130, keeps load time at around 30 min

stocks = ticker_list[14]
stocks

'SMCI'

In [5]:
# Now we must create a dataframe to store all of our stock data, as well as
# express the stocks that we wish to research
statements = {}
# stocks = ["ENPH", "IBM", "MBUU", "MANH", "TREX", "LSCC"]

# state the different statement types we want to cycle through
statement_types =  ["INCOME_STATEMENT", "BALANCE_SHEET", "CASH_FLOW"] #,"EARNINGS"

# Note that because of API limitations, we are only allowed to make 5 API
# requests per minute with our free version. 

# Future versions of this code
# should make a parquay of previous data, so that we can use that for past
# and future valuations without needing to redownload the data. This hopefully
# will also be usable for a Discounted Cash Flow model to be created in this
# format, which would also export to pdf for easy access to graphs, financial
# analysis which is automated for analysts, and an easy-to-read methodology
# that fixes many of the issues with excel.

for stock in stocks:
    statements[stock] = {}
    for statement_type in statement_types:
        statement_key = statement_type.split("_")[0]
        statements[stock][statement_key] = {}
        try:
            fin_statement = get_fin_stmt(stock, statement_type)
            for annual_data in fin_statement:
                date = annual_data["fiscalDateEnding"]

                statements[stock][statement_key][date] = annual_data
        except:
            print("Error retrieving " + statement_type + " for " + stock)


Error: No INCOME_STATEMENT data found for I
Error retrieving INCOME_STATEMENT for I
Error: No BALANCE_SHEET data found for I
Error retrieving BALANCE_SHEET for I
Error: No CASH_FLOW data found for I
Error retrieving CASH_FLOW for I


In [6]:
def calc_ROIC(EBIT, Taxes, Dividends, Debt, Equity, Cash_Equiv):
    if EBIT != "None":
        try:
            NOPAT = (int(EBIT) - int(Taxes)) - int(Dividends)
            Invested_Capital = int(Debt) + int(Equity) - int(Cash_Equiv)
            ROIC = round(NOPAT / Invested_Capital, 2)
        except:
            NOPAT = (int(EBIT) - int(Taxes))
            Invested_Capital = int(Debt) + int(Equity) - int(Cash_Equiv)
            ROIC = round(NOPAT / Invested_Capital, 2)
    else: ROIC = 0
    return ROIC

In [7]:
def calc_EPS(Net_Income, Dividends, Shares):
    #try:
    #    EPS = round((int(Net_Income) - int(Dividends)) / int(Shares),2)
    #except:
    try:
        EPS = round((int(Net_Income)) / int(Shares),2)
    except:
        EPS = 0
    return EPS

In [8]:
def calc_Current_Ratio(Current_Assets, Current_Liabilities):
    try:
        Current_Ratio = round(int(Current_Assets) / int(Current_Liabilities),2)
    except:
        Current_Ratio = 0
    return Current_Ratio

In [9]:
def EPS_Valuation(EPS, ROIC):
    try:
        Multiple = (8.5 + 60*(ROIC))
        Price = round(Multiple * EPS,2)
    except:
        Price = 0
    return Price

In [12]:
# right now we are removing dividends from the EPS calculation since I will be adding dividend yields into the 
# valuation later on if I have time. Note: underestimating multiples for some companies, but strongly on dividend companies
for stock in statements:
    statements[stock]["VALUATION"] = {}
    for date in statements[stock]["INCOME"]:
            statements[stock]["VALUATION"][date] = {}
            new = statements[stock]["VALUATION"][date]
            old = statements[stock]
            # create ROIC values in Valuation
            new["ROIC"] = calc_ROIC(EBIT = old["INCOME"][date]["ebit"], 
                                    Taxes = old["INCOME"][date]["incomeTaxExpense"],
                                    Dividends = old["CASH"][date]["dividendPayout"],
                                    Debt = old["BALANCE"][date]["totalLiabilities"],
                                   Equity = old["BALANCE"][date]["totalShareholderEquity"],
                                   Cash_Equiv = old["BALANCE"][date]["cashAndShortTermInvestments"])
            
            # create EPS values in Valuation
            new["EPS"] = calc_EPS(Net_Income = old["INCOME"][date]["netIncome"], 
                                   Dividends = old["CASH"][date]["dividendPayout"],
                                  Shares = old["BALANCE"][date]["commonStockSharesOutstanding"])
            
            # create current ratio values in valuation
            new["Current_Ratio"] = calc_Current_Ratio(Current_Assets = old["BALANCE"][date]["totalCurrentAssets"],
                                                      Current_Liabilities = old["BALANCE"][date]["totalCurrentLiabilities"])
            new["EPS_Valuation"] = EPS_Valuation(EPS = new["EPS"],
                                                ROIC = new["ROIC"])

KeyError: '2018-12-31'

In [ ]:
# use keys to make path, then use os to save path name
# forget os.walk
# !pip install parquet
# ! pip install pyarrow

if not os.path.exists("Statement_Data"):
    os.mkdir("Statement_Data")
for stock in statements:
    if not os.path.exists("Statement_Data/" + stock):
        os.mkdir("Statement_Data/" + stock)
    for statement_type in statements[stock]:
        if not os.path.exists("Statement_Data/" + stock + '/' + statement_type):
            os.mkdir("Statement_Data/" + stock + '/' + statement_type)
        pd.DataFrame(statements[stock][statement_type]).to_parquet("Statement_Data/" + stock + "/" + statement_type + '/data.parquet')